# Steam Store Analytics - Snowflake SQL Data Warehouse Project

End-to-end Snowflake data warehouse for 27K+ Steam games using medallion architecture (Bronze/Silver/Gold), dimensional modeling, and Snowflake-native AI features.

## Bronze Layer: Raw Ingestion
- COPY INTO from staged Steam dataset
- Minimal transformations

In [ ]:
USE ROLE ROLE_TEAM_TRIPLEJ;
USE WAREHOUSE ANIMAL_TASK_WH;
USE DATABASE DB_TEAM_TRIPLEJ;

#### Create 3 Schemas: Bronze, Silver, Gold

In [ ]:
CREATE SCHEMA IF NOT EXISTS Bronze;
CREATE SCHEMA IF NOT EXISTS Silver;
CREATE SCHEMA IF NOT EXISTS Gold;

#### Import selected datasets into bronze layer

In [ ]:
-- Create Bronze Stage with CSV format
CREATE STAGE IF NOT EXISTS BRONZE_STAGE;
CREATE OR REPLACE FILE FORMAT FF_CSV_STANDARD
  TYPE = CSV
  PARSE_HEADER = TRUE
  FIELD_OPTIONALLY_ENCLOSED_BY = '"'
  EMPTY_FIELD_AS_NULL = TRUE;

In [ ]:
-- Use Bronze Schema
USE SCHEMA BRONZE;

-- Create file format

-- Create Bronze Tables via Schema interface
CREATE OR REPLACE TABLE Bronze.steam_games
USING TEMPLATE (
    SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) WITHIN GROUP (ORDER BY order_id)
    FROM TABLE(
        INFER_SCHEMA(
            LOCATION => '@DB_TEAM_TRIPLEJ.BRONZE.BRONZE_STAGE/steam.csv',
            FILE_FORMAT => 'FF_CSV_STANDARD'
        )
    )
);

CREATE OR REPLACE TABLE Bronze.steam_descriptions
USING TEMPLATE (
    SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) WITHIN GROUP (ORDER BY order_id)
    FROM TABLE(
        INFER_SCHEMA(
              LOCATION => '@DB_TEAM_TRIPLEJ.BRONZE.BRONZE_STAGE/steam_description_data.csv',
              FILE_FORMAT => 'FF_CSV_STANDARD'
        )
    )
);

In [ ]:
-- Load Bronze
COPY INTO BRONZE.STEAM_GAMES
FROM @BRONZE_STAGE/steam.csv
FILE_FORMAT = (FORMAT_NAME = 'FF_CSV_STANDARD')
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

COPY INTO BRONZE.STEAM_DESCRIPTIONS
FROM @BRONZE_STAGE/steam_description_data.csv
FILE_FORMAT = (FORMAT_NAME = 'FF_CSV_STANDARD')
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

In [ ]:
-- Bronze Steam Games Verification
SELECT * FROM BRONZE.STEAM_GAMES LIMIT 10;

In [ ]:
-- Bronze Steam Descriptions Verification
SELECT * FROM BRONZE.STEAM_DESCRIPTIONS LIMIT 10;

## Silver Layer: Cleaning & Normalization
- Deduplication
- Text normalization
- Bridge table construction

#### Design Dimension, Facts, and Composite Tables for Silver layer

ie. Game Dimension, Developer Dimension, Publisher Dimension, Category dimension, Description Dimension, Fact table with foreign keys

#### Create Silver layer schema objects

In [ ]:
USE SCHEMA SILVER;

-- Developer dimension
CREATE OR REPLACE TABLE DIM_DEVELOPER (
    DEVELOPER_KEY INT AUTOINCREMENT,
    DEVELOPER VARCHAR(500),
    PRIMARY KEY (DEVELOPER_KEY)
);

-- Publisher dimension
CREATE OR REPLACE TABLE DIM_PUBLISHER (
    PUBLISHER_KEY INT AUTOINCREMENT,
    PUBLISHER VARCHAR(500),
    PRIMARY KEY (PUBLISHER_KEY)
);

-- Category dimension
CREATE OR REPLACE TABLE DIM_CATEGORY (
    CATEGORY_KEY INT AUTOINCREMENT,
    CATEGORY VARCHAR(200),
    PRIMARY KEY (CATEGORY_KEY)
);

-- Genre dimension
CREATE OR REPLACE TABLE DIM_GENRE (
    GENRE_KEY INT AUTOINCREMENT,
    GENRE VARCHAR(200),
    PRIMARY KEY (GENRE_KEY)
);

-- Description dimension
CREATE OR REPLACE TABLE DIM_DESCRIPTION (
    APP_ID NUMBER,
    DETAILED_DESCRIPTION VARCHAR(16777216),
    ABOUT_THE_GAME VARCHAR(16777216),
    SHORT_DESCRIPTION VARCHAR(16777216),
    PRIMARY KEY (APP_ID)
);

-- Game dimension
CREATE OR REPLACE TABLE DIM_GAME (
    APP_ID NUMBER,
    NAME VARCHAR(500),
    RELEASE_DATE DATE,
    REQUIRED_AGE NUMBER,
    PLATFORMS VARCHAR(100),
    ENGLISH NUMBER,
    PRIMARY KEY (APP_ID)
);

-- Fact table (CORRECTED with all metrics columns)
CREATE OR REPLACE TABLE FACT_GAME_METRICS (
    FACT_KEY INT AUTOINCREMENT,
    APP_ID NUMBER,
    DEVELOPER_KEY INT,
    PUBLISHER_KEY INT,
    PRICE NUMBER(10,2),
    POSITIVE_RATINGS NUMBER,
    NEGATIVE_RATINGS NUMBER,
    AVERAGE_PLAYTIME NUMBER,
    MEDIAN_PLAYTIME NUMBER,
    OWNERS VARCHAR(100),
    ACHIEVEMENTS NUMBER,
    PRIMARY KEY (FACT_KEY),
    FOREIGN KEY (APP_ID) REFERENCES DIM_GAME(APP_ID),
    FOREIGN KEY (DEVELOPER_KEY) REFERENCES DIM_DEVELOPER(DEVELOPER_KEY),
    FOREIGN KEY (PUBLISHER_KEY) REFERENCES DIM_PUBLISHER(PUBLISHER_KEY)
);

-- Bridge table for Game-Category (many-to-many)
CREATE OR REPLACE TABLE BRIDGE_GAME_CATEGORY (
    APP_ID NUMBER,
    CATEGORY_KEY INT,
    FOREIGN KEY (APP_ID) REFERENCES DIM_GAME(APP_ID),
    FOREIGN KEY (CATEGORY_KEY) REFERENCES DIM_CATEGORY(CATEGORY_KEY)
);

-- Bridge table for Game-Genre (many-to-many)
CREATE OR REPLACE TABLE BRIDGE_GAME_GENRE (
    APP_ID NUMBER,
    GENRE_KEY INT,
    FOREIGN KEY (APP_ID) REFERENCES DIM_GAME(APP_ID),
    FOREIGN KEY (GENRE_KEY) REFERENCES DIM_GENRE(GENRE_KEY)
);

#### Populate silver layer data from bronze layer maintaining data consistency

In [ ]:
USE SCHEMA SILVER;

-- Clean + Load Developer Dimension
TRUNCATE TABLE SILVER.DIM_DEVELOPER;
INSERT INTO SILVER.DIM_DEVELOPER (DEVELOPER)
SELECT DISTINCT
    INITCAP(TRIM("developer")) AS DEVELOPER
FROM BRONZE.STEAM_GAMES
WHERE "developer" IS NOT NULL
  AND TRIM("developer") <> ''
  AND UPPER(TRIM("developer")) NOT IN ('N/A', 'UNKNOWN', 'NONE', '');

-- Clean + Load Publisher Dimension
TRUNCATE TABLE SILVER.DIM_PUBLISHER;
INSERT INTO SILVER.DIM_PUBLISHER (PUBLISHER)
SELECT DISTINCT
    INITCAP(TRIM("publisher")) AS PUBLISHER
FROM BRONZE.STEAM_GAMES
WHERE "publisher" IS NOT NULL
  AND TRIM("publisher") <> ''
  AND UPPER(TRIM("publisher")) NOT IN ('N/A', 'UNKNOWN', 'NONE', '');

-- Clean + Load Category Dimension
TRUNCATE TABLE SILVER.DIM_CATEGORY;
INSERT INTO SILVER.DIM_CATEGORY (CATEGORY)
SELECT DISTINCT
    INITCAP(TRIM(cat.value)) AS CATEGORY
FROM BRONZE.STEAM_GAMES,
     LATERAL SPLIT_TO_TABLE("categories", ';') AS cat
WHERE cat.value IS NOT NULL
  AND TRIM(cat.value) <> ''
ORDER BY CATEGORY;

-- Clean + Load Genre Dimension
TRUNCATE TABLE SILVER.DIM_GENRE;
INSERT INTO SILVER.DIM_GENRE (GENRE)
SELECT DISTINCT
    INITCAP(TRIM(gen.value)) AS GENRE
FROM BRONZE.STEAM_GAMES,
     LATERAL SPLIT_TO_TABLE("genres", ';') AS gen
WHERE gen.value IS NOT NULL
  AND TRIM(gen.value) <> ''
ORDER BY GENRE;

-- Clean + Load Description Dimension
TRUNCATE TABLE SILVER.DIM_DESCRIPTION;
INSERT INTO SILVER.DIM_DESCRIPTION (APP_ID, DETAILED_DESCRIPTION, ABOUT_THE_GAME, SHORT_DESCRIPTION)
SELECT DISTINCT
    "steam_appid" AS APP_ID,
    TRIM("detailed_description") AS DETAILED_DESCRIPTION,
    TRIM("about_the_game") AS ABOUT_THE_GAME,
    TRIM("short_description") AS SHORT_DESCRIPTION
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE "steam_appid" IS NOT NULL;

-- Clean + Load Game Dimension
TRUNCATE TABLE SILVER.DIM_GAME;
INSERT INTO SILVER.DIM_GAME (APP_ID, NAME, RELEASE_DATE, REQUIRED_AGE, PLATFORMS, ENGLISH)
SELECT DISTINCT
    "appid" AS APP_ID,
    INITCAP(TRIM("name")) AS NAME,
    "release_date" AS RELEASE_DATE,
    NULLIF("required_age", 0) AS REQUIRED_AGE,
    LOWER(TRIM("platforms")) AS PLATFORMS,
    CASE WHEN "english" = 1 THEN 1 ELSE 0 END AS ENGLISH
FROM BRONZE.STEAM_GAMES
WHERE "appid" IS NOT NULL
  AND TRIM("name") <> '';

-- Clean + Build Fact Table
TRUNCATE TABLE SILVER.FACT_GAME_METRICS;
INSERT INTO SILVER.FACT_GAME_METRICS (
    APP_ID, 
    DEVELOPER_KEY, 
    PUBLISHER_KEY,
    PRICE, 
    POSITIVE_RATINGS, 
    NEGATIVE_RATINGS,
    AVERAGE_PLAYTIME, 
    MEDIAN_PLAYTIME, 
    OWNERS,
    ACHIEVEMENTS
)
SELECT 
    g."appid" AS APP_ID,
    dev.DEVELOPER_KEY,
    pub.PUBLISHER_KEY,
    CASE WHEN g."price" >= 0 THEN g."price" ELSE NULL END AS PRICE,
    CASE WHEN g."positive_ratings" >= 0 THEN g."positive_ratings" ELSE NULL END AS POSITIVE_RATINGS,
    CASE WHEN g."negative_ratings" >= 0 THEN g."negative_ratings" ELSE NULL END AS NEGATIVE_RATINGS,
    CASE WHEN g."average_playtime" >= 0 THEN g."average_playtime" ELSE NULL END AS AVERAGE_PLAYTIME,
    CASE WHEN g."median_playtime" >= 0 THEN g."median_playtime" ELSE NULL END AS MEDIAN_PLAYTIME,
    TRIM(g."owners") AS OWNERS,
    CASE WHEN g."achievements" >= 0 THEN g."achievements" ELSE NULL END AS ACHIEVEMENTS
FROM BRONZE.STEAM_GAMES g
LEFT JOIN SILVER.DIM_DEVELOPER dev 
    ON INITCAP(TRIM(g."developer")) = dev.DEVELOPER
LEFT JOIN SILVER.DIM_PUBLISHER pub 
    ON INITCAP(TRIM(g."publisher")) = pub.PUBLISHER
WHERE g."appid" IS NOT NULL;

-- Populate Bridge Table: Game-Category
TRUNCATE TABLE SILVER.BRIDGE_GAME_CATEGORY;
INSERT INTO SILVER.BRIDGE_GAME_CATEGORY (APP_ID, CATEGORY_KEY)
SELECT DISTINCT
    g."appid" AS APP_ID,
    cat.CATEGORY_KEY
FROM BRONZE.STEAM_GAMES g,
     LATERAL SPLIT_TO_TABLE(g."categories", ';') AS cat_split
INNER JOIN SILVER.DIM_CATEGORY cat 
    ON INITCAP(TRIM(cat_split.value)) = cat.CATEGORY
WHERE g."appid" IS NOT NULL
  AND cat_split.value IS NOT NULL
  AND TRIM(cat_split.value) <> '';

-- Populate Bridge Table: Game-Genre
TRUNCATE TABLE SILVER.BRIDGE_GAME_GENRE;
INSERT INTO SILVER.BRIDGE_GAME_GENRE (APP_ID, GENRE_KEY)
SELECT DISTINCT
    g."appid" AS APP_ID,
    gen.GENRE_KEY
FROM BRONZE.STEAM_GAMES g,
     LATERAL SPLIT_TO_TABLE(g."genres", ';') AS gen_split
INNER JOIN SILVER.DIM_GENRE gen 
    ON INITCAP(TRIM(gen_split.value)) = gen.GENRE
WHERE g."appid" IS NOT NULL
  AND gen_split.value IS NOT NULL
  AND TRIM(gen_split.value) <> '';

-- Preview with joins
SELECT 
    f.APP_ID,
    g.NAME,
    dev.DEVELOPER,
    pub.PUBLISHER,
    f.PRICE,
    f.POSITIVE_RATINGS,
    f.OWNERS
FROM SILVER.FACT_GAME_METRICS f
JOIN SILVER.DIM_GAME g ON f.APP_ID = g.APP_ID
LEFT JOIN SILVER.DIM_DEVELOPER dev ON f.DEVELOPER_KEY = dev.DEVELOPER_KEY
LEFT JOIN SILVER.DIM_PUBLISHER pub ON f.PUBLISHER_KEY = pub.PUBLISHER_KEY
LIMIT 20;

## Gold Layer: Analytics-Ready Tables
- Fact tables
- Dimension tables
- Aggregations

#### Defining 3 use cases for business reporting

Each of these tables is designed to give aggregated metrics on average playtime, reviews, and total sales:
    
i.) Category Level - Useful for publishers who want to see what categories (e.g., MMO, VR Supported, Multiplayer, etc.) are performing well, so they can focus on them when prioritizing future games.

ii.) Genre level - This aggregated table functions similarly to the category level, but functions at the genre level (e.g., Horror, Fantasy, Sports), which is also useful for prioritizing future game development.

iii.) Publisher Level - This table allows game publishers to see how their own playtime, review, and sales stats compare to their competitors. It helps answer key questions such as "Are we having a poor performing year, or is this an industry trend that our competitors are facing as well?"

#### Design and populate the Gold layer tables to meet those use cases


In [ ]:
CREATE OR REPLACE TABLE GOLD.CATEGORY_METRICS AS

SELECT 
    CATEGORY,
    AVG(AVERAGE_PLAYTIME) AS AVERAGE_CAT_PLAYTIME,
    AVG(
        CASE 
            WHEN NEGATIVE_RATINGS = 0 THEN NULL
            ELSE POSITIVE_RATINGS / NEGATIVE_RATINGS
        END
    ) AS AVERAGE_REVIEW,
    -- Approximation of sales
    SUM(TO_NUMBER(LEFT(OWNERS, POSITION('-' IN OWNERS) - 1))) AS TOTAL_SALES

FROM SILVER.FACT_GAME_METRICS f
    JOIN SILVER.DIM_GAME g ON f.APP_ID = g.APP_ID
    JOIN SILVER.BRIDGE_GAME_CATEGORY b ON g.APP_ID = b.APP_ID
    JOIN SILVER.DIM_CATEGORY c ON b.CATEGORY_KEY = c.CATEGORY_KEY

GROUP BY CATEGORY;

In [ ]:
CREATE OR REPLACE TABLE GOLD.GENRE_METRICS AS 

SELECT 
    GENRE,
    AVG(AVERAGE_PLAYTIME) AS AVERAGE_GENRE_PLAYTIME,
    AVG(
        CASE 
            WHEN NEGATIVE_RATINGS = 0 THEN NULL
            ELSE POSITIVE_RATINGS / NEGATIVE_RATINGS
        END
    ) AS AVERAGE_REVIEW,
    -- Approximation of sales
    SUM(TO_NUMBER(LEFT(OWNERS, POSITION('-' IN OWNERS) - 1))) AS TOTAL_SALES

FROM SILVER.FACT_GAME_METRICS f
    JOIN SILVER.DIM_GAME g ON f.APP_ID = g.APP_ID
    JOIN SILVER.BRIDGE_GAME_GENRE b ON g.APP_ID = b.APP_ID
    JOIN SILVER.DIM_GENRE dg ON b.GENRE_KEY = dg.GENRE_KEY

GROUP BY GENRE;

In [ ]:
CREATE OR REPLACE TABLE GOLD.PUBLISHER_METRICS AS

SELECT 
    PUBLISHER,
    AVG(AVERAGE_PLAYTIME) AS AVERAGE_PUBLISHER_PLAYTIME,
    AVG(
        CASE 
            WHEN NEGATIVE_RATINGS = 0 THEN NULL
            ELSE POSITIVE_RATINGS / NEGATIVE_RATINGS
        END
    ) AS AVERAGE_REVIEW,
    -- Approximation of sales
    SUM(TO_NUMBER(LEFT(OWNERS, POSITION('-' IN OWNERS) - 1))) AS TOTAL_SALES

FROM SILVER.FACT_GAME_METRICS f
    JOIN SILVER.DIM_PUBLISHER p ON f.PUBLISHER_KEY = p.PUBLISHER_KEY

GROUP BY PUBLISHER;

In [ ]:
SELECT * FROM GOLD.GENRE_METRICS;

## Incremental Loads
- Snowflake Streams
- Upsert logic

Create a small data file which is of the same format/columns as the original data file but only a handful of rows, and hand populate it with some new data to simulate new incoming data

Load this incremental data automatically into Bronze, Silver and Gold layers.


In [ ]:
-- This number was 27075 before uploading

USE SCHEMA SILVER;
SELECT COUNT(APP_ID) FROM FACT_GAME_METRICS;

In [ ]:
-- Create streams to track changes in Bronze tables
CREATE OR REPLACE STREAM BRONZE.BRONZE_STEAM_GAMES_STREAM 
ON TABLE BRONZE.STEAM_GAMES;

CREATE OR REPLACE STREAM BRONZE.BRONZE_STEAM_DESCRIPTIONS_STREAM 
ON TABLE BRONZE.STEAM_DESCRIPTIONS;


In [ ]:
USE SCHEMA BRONZE;

-- Load Bronze
COPY INTO BRONZE.STEAM_GAMES
FROM @BRONZE_STAGE/steam_games_test_2.csv
FILE_FORMAT = (FORMAT_NAME = 'FF_CSV_STANDARD')
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

COPY INTO BRONZE.STEAM_DESCRIPTIONS
FROM @BRONZE_STAGE/steam_description_test_2.csv
FILE_FORMAT = (FORMAT_NAME = 'FF_CSV_STANDARD')
MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE;

In [ ]:
-- Move updated data from the bronze stream to the silver layer
USE SCHEMA SILVER;

-- Materialize stream data into temp tables FIRST
CREATE OR REPLACE TEMPORARY TABLE temp_stream_games AS
SELECT * FROM BRONZE.BRONZE_STEAM_GAMES_STREAM
WHERE METADATA$ACTION = 'INSERT';

CREATE OR REPLACE TEMPORARY TABLE temp_stream_descriptions AS
SELECT * FROM BRONZE.BRONZE_STEAM_DESCRIPTIONS_STREAM
WHERE METADATA$ACTION = 'INSERT';

-- Now use the temp tables instead of the streams

-- Add new developers (only if they don't already exist)
INSERT INTO SILVER.DIM_DEVELOPER (DEVELOPER)
SELECT DISTINCT
    INITCAP(TRIM("developer")) AS DEVELOPER
FROM temp_stream_games
WHERE "developer" IS NOT NULL
  AND TRIM("developer") <> ''
  AND UPPER(TRIM("developer")) NOT IN ('N/A', 'UNKNOWN', 'NONE', '')
  AND INITCAP(TRIM("developer")) NOT IN (SELECT DEVELOPER FROM SILVER.DIM_DEVELOPER);

-- Add new publishers (only if they don't already exist)
INSERT INTO SILVER.DIM_PUBLISHER (PUBLISHER)
SELECT DISTINCT
    INITCAP(TRIM("publisher")) AS PUBLISHER
FROM temp_stream_games
WHERE "publisher" IS NOT NULL
  AND TRIM("publisher") <> ''
  AND UPPER(TRIM("publisher")) NOT IN ('N/A', 'UNKNOWN', 'NONE', '')
  AND INITCAP(TRIM("publisher")) NOT IN (SELECT PUBLISHER FROM SILVER.DIM_PUBLISHER);

-- Add new categories (only if they don't already exist)
INSERT INTO SILVER.DIM_CATEGORY (CATEGORY)
SELECT DISTINCT
    INITCAP(TRIM(cat.value)) AS CATEGORY
FROM temp_stream_games,
     LATERAL SPLIT_TO_TABLE("categories", ';') AS cat
WHERE cat.value IS NOT NULL
  AND TRIM(cat.value) <> ''
  AND INITCAP(TRIM(cat.value)) NOT IN (SELECT CATEGORY FROM SILVER.DIM_CATEGORY);

-- Add new genres (only if they don't already exist)
INSERT INTO SILVER.DIM_GENRE (GENRE)
SELECT DISTINCT
    INITCAP(TRIM(gen.value)) AS GENRE
FROM temp_stream_games,
     LATERAL SPLIT_TO_TABLE("genres", ';') AS gen
WHERE gen.value IS NOT NULL
  AND TRIM(gen.value) <> ''
  AND INITCAP(TRIM(gen.value)) NOT IN (SELECT GENRE FROM SILVER.DIM_GENRE);

-- Upsert descriptions (insert new, update existing)
MERGE INTO SILVER.DIM_DESCRIPTION AS target
USING (
    SELECT DISTINCT
        "steam_appid" AS APP_ID,
        TRIM("detailed_description") AS DETAILED_DESCRIPTION,
        TRIM("about_the_game") AS ABOUT_THE_GAME,
        TRIM("short_description") AS SHORT_DESCRIPTION
    FROM temp_stream_descriptions
    WHERE "steam_appid" IS NOT NULL
) AS source
ON target.APP_ID = source.APP_ID
WHEN MATCHED THEN
    UPDATE SET
        DETAILED_DESCRIPTION = source.DETAILED_DESCRIPTION,
        ABOUT_THE_GAME = source.ABOUT_THE_GAME,
        SHORT_DESCRIPTION = source.SHORT_DESCRIPTION
WHEN NOT MATCHED THEN
    INSERT (APP_ID, DETAILED_DESCRIPTION, ABOUT_THE_GAME, SHORT_DESCRIPTION)
    VALUES (source.APP_ID, source.DETAILED_DESCRIPTION, source.ABOUT_THE_GAME, source.SHORT_DESCRIPTION);

-- Upsert games (insert new, update existing)
MERGE INTO SILVER.DIM_GAME AS target
USING (
    SELECT DISTINCT
        "appid" AS APP_ID,
        INITCAP(TRIM("name")) AS NAME,
        "release_date" AS RELEASE_DATE,
        NULLIF("required_age", 0) AS REQUIRED_AGE,
        LOWER(TRIM("platforms")) AS PLATFORMS,
        CASE WHEN "english" = 1 THEN 1 ELSE 0 END AS ENGLISH
    FROM temp_stream_games
    WHERE "appid" IS NOT NULL
      AND TRIM("name") <> ''
) AS source
ON target.APP_ID = source.APP_ID
WHEN MATCHED THEN
    UPDATE SET
        NAME = source.NAME,
        RELEASE_DATE = source.RELEASE_DATE,
        REQUIRED_AGE = source.REQUIRED_AGE,
        PLATFORMS = source.PLATFORMS,
        ENGLISH = source.ENGLISH
WHEN NOT MATCHED THEN
    INSERT (APP_ID, NAME, RELEASE_DATE, REQUIRED_AGE, PLATFORMS, ENGLISH)
    VALUES (source.APP_ID, source.NAME, source.RELEASE_DATE, source.REQUIRED_AGE, source.PLATFORMS, source.ENGLISH);

-- Upsert fact metrics (insert new, update existing)
MERGE INTO SILVER.FACT_GAME_METRICS AS target
USING (
    SELECT 
        g."appid" AS APP_ID,
        dev.DEVELOPER_KEY,
        pub.PUBLISHER_KEY,
        CASE WHEN g."price" >= 0 THEN g."price" ELSE NULL END AS PRICE,
        CASE WHEN g."positive_ratings" >= 0 THEN g."positive_ratings" ELSE NULL END AS POSITIVE_RATINGS,
        CASE WHEN g."negative_ratings" >= 0 THEN g."negative_ratings" ELSE NULL END AS NEGATIVE_RATINGS,
        CASE WHEN g."average_playtime" >= 0 THEN g."average_playtime" ELSE NULL END AS AVERAGE_PLAYTIME,
        CASE WHEN g."median_playtime" >= 0 THEN g."median_playtime" ELSE NULL END AS MEDIAN_PLAYTIME,
        TRIM(g."owners") AS OWNERS,
        CASE WHEN g."achievements" >= 0 THEN g."achievements" ELSE NULL END AS ACHIEVEMENTS
    FROM temp_stream_games g
    LEFT JOIN SILVER.DIM_DEVELOPER dev 
        ON INITCAP(TRIM(g."developer")) = dev.DEVELOPER
    LEFT JOIN SILVER.DIM_PUBLISHER pub 
        ON INITCAP(TRIM(g."publisher")) = pub.PUBLISHER
    WHERE g."appid" IS NOT NULL
) AS source
ON target.APP_ID = source.APP_ID
WHEN MATCHED THEN
    UPDATE SET
        DEVELOPER_KEY = source.DEVELOPER_KEY,
        PUBLISHER_KEY = source.PUBLISHER_KEY,
        PRICE = source.PRICE,
        POSITIVE_RATINGS = source.POSITIVE_RATINGS,
        NEGATIVE_RATINGS = source.NEGATIVE_RATINGS,
        AVERAGE_PLAYTIME = source.AVERAGE_PLAYTIME,
        MEDIAN_PLAYTIME = source.MEDIAN_PLAYTIME,
        OWNERS = source.OWNERS,
        ACHIEVEMENTS = source.ACHIEVEMENTS
WHEN NOT MATCHED THEN
    INSERT (APP_ID, DEVELOPER_KEY, PUBLISHER_KEY, PRICE, POSITIVE_RATINGS, NEGATIVE_RATINGS, 
            AVERAGE_PLAYTIME, MEDIAN_PLAYTIME, OWNERS, ACHIEVEMENTS)
    VALUES (source.APP_ID, source.DEVELOPER_KEY, source.PUBLISHER_KEY, source.PRICE, 
            source.POSITIVE_RATINGS, source.NEGATIVE_RATINGS, source.AVERAGE_PLAYTIME, 
            source.MEDIAN_PLAYTIME, source.OWNERS, source.ACHIEVEMENTS);

-- Refresh bridge tables (delete old relationships for updated games, insert current ones)
DELETE FROM SILVER.BRIDGE_GAME_CATEGORY
WHERE APP_ID IN (
    SELECT "appid" 
    FROM temp_stream_games 
    WHERE "appid" IS NOT NULL
);

INSERT INTO SILVER.BRIDGE_GAME_CATEGORY (APP_ID, CATEGORY_KEY)
SELECT DISTINCT
    g."appid" AS APP_ID,
    cat.CATEGORY_KEY
FROM temp_stream_games g,
     LATERAL SPLIT_TO_TABLE(g."categories", ';') AS cat_split
INNER JOIN SILVER.DIM_CATEGORY cat 
    ON INITCAP(TRIM(cat_split.value)) = cat.CATEGORY
WHERE g."appid" IS NOT NULL
  AND cat_split.value IS NOT NULL
  AND TRIM(cat_split.value) <> '';

DELETE FROM SILVER.BRIDGE_GAME_GENRE
WHERE APP_ID IN (
    SELECT "appid" 
    FROM temp_stream_games 
    WHERE "appid" IS NOT NULL
);

INSERT INTO SILVER.BRIDGE_GAME_GENRE (APP_ID, GENRE_KEY)
SELECT DISTINCT
    g."appid" AS APP_ID,
    gen.GENRE_KEY
FROM temp_stream_games g,
     LATERAL SPLIT_TO_TABLE(g."genres", ';') AS gen_split
INNER JOIN SILVER.DIM_GENRE gen 
    ON INITCAP(TRIM(gen_split.value)) = gen.GENRE
WHERE g."appid" IS NOT NULL
  AND gen_split.value IS NOT NULL
  AND TRIM(gen_split.value) <> '';

In [ ]:
-- Verify that data has been updated into silver

-- Check stream is now empty (consumed)
SELECT COUNT(*) FROM BRONZE.BRONZE_STEAM_GAMES_STREAM;  -- Should return 0

-- Preview updated Silver data
SELECT 
    f.APP_ID,
    g.NAME,
    dev.DEVELOPER,
    pub.PUBLISHER,
    f.PRICE,
    f.POSITIVE_RATINGS,
    f.OWNERS
FROM SILVER.FACT_GAME_METRICS f
JOIN SILVER.DIM_GAME g ON f.APP_ID = g.APP_ID
LEFT JOIN SILVER.DIM_DEVELOPER dev ON f.DEVELOPER_KEY = dev.DEVELOPER_KEY
LEFT JOIN SILVER.DIM_PUBLISHER pub ON f.PUBLISHER_KEY = pub.PUBLISHER_KEY
ORDER BY f.APP_ID DESC
LIMIT 20;

In [ ]:
-- Refresh the gold layer

TRUNCATE TABLE GOLD.CATEGORY_METRICS;
INSERT INTO GOLD.CATEGORY_METRICS
SELECT 
    CATEGORY,
    AVG(AVERAGE_PLAYTIME) AS AVERAGE_CAT_PLAYTIME,
    AVG(
        CASE 
            WHEN NEGATIVE_RATINGS = 0 THEN NULL
            ELSE POSITIVE_RATINGS / NEGATIVE_RATINGS
        END
    ) AS AVERAGE_REVIEW,
    -- Approximation of sales
    SUM(TO_NUMBER(LEFT(OWNERS, POSITION('-' IN OWNERS) - 1))) AS TOTAL_SALES
FROM SILVER.FACT_GAME_METRICS f
    JOIN SILVER.DIM_GAME g ON f.APP_ID = g.APP_ID
    JOIN SILVER.BRIDGE_GAME_CATEGORY b ON g.APP_ID = b.APP_ID
    JOIN SILVER.DIM_CATEGORY c ON b.CATEGORY_KEY = c.CATEGORY_KEY
GROUP BY CATEGORY;

-- Refresh Genre Metrics
TRUNCATE TABLE GOLD.GENRE_METRICS;
INSERT INTO GOLD.GENRE_METRICS
SELECT 
    GENRE,
    AVG(AVERAGE_PLAYTIME) AS AVERAGE_CAT_PLAYTIME,
    AVG(
        CASE 
            WHEN NEGATIVE_RATINGS = 0 THEN NULL
            ELSE POSITIVE_RATINGS / NEGATIVE_RATINGS
        END
    ) AS AVERAGE_REVIEW,
    -- Approximation of sales
    SUM(TO_NUMBER(LEFT(OWNERS, POSITION('-' IN OWNERS) - 1))) AS TOTAL_SALES
FROM SILVER.FACT_GAME_METRICS f
    JOIN SILVER.DIM_GAME g ON f.APP_ID = g.APP_ID
    JOIN SILVER.BRIDGE_GAME_GENRE b ON g.APP_ID = b.APP_ID
    JOIN SILVER.DIM_GENRE dg ON b.GENRE_KEY = dg.GENRE_KEY
GROUP BY GENRE;

-- Refresh Publisher Metrics
TRUNCATE TABLE GOLD.PUBLISHER_METRICS;
INSERT INTO GOLD.PUBLISHER_METRICS
SELECT 
    PUBLISHER,
    AVG(AVERAGE_PLAYTIME) AS AVERAGE_CAT_PLAYTIME,
    AVG(
        CASE 
            WHEN NEGATIVE_RATINGS = 0 THEN NULL
            ELSE POSITIVE_RATINGS / NEGATIVE_RATINGS
        END
    ) AS AVERAGE_REVIEW,
    -- Approximation of sales
    SUM(TO_NUMBER(LEFT(OWNERS, POSITION('-' IN OWNERS) - 1))) AS TOTAL_SALES
FROM SILVER.FACT_GAME_METRICS f
    JOIN SILVER.DIM_PUBLISHER p ON f.PUBLISHER_KEY = p.PUBLISHER_KEY
GROUP BY PUBLISHER;

In [ ]:
-- Test case for Q.10
-- Should be 10 higher than the orginal select statement

USE SCHEMA SILVER;
SELECT COUNT(APP_ID) FROM FACT_GAME_METRICS;

In [ ]:
-- Verification for Silver Layer
SELECT 'DIM_DEVELOPER' AS TABLE_NAME, COUNT(*) AS ROW_COUNT FROM SILVER.DIM_DEVELOPER
UNION ALL
SELECT 'DIM_PUBLISHER', COUNT(*) FROM SILVER.DIM_PUBLISHER
UNION ALL
SELECT 'FACT_GAME_METRICS', COUNT(*) FROM SILVER.FACT_GAME_METRICS
UNION ALL
SELECT 'BRIDGE_GAME_CATEGORY', COUNT(*) FROM SILVER.BRIDGE_GAME_CATEGORY
UNION ALL
SELECT 'BRIDGE_GAME_GENRE', COUNT(*) FROM SILVER.BRIDGE_GAME_GENRE;

## AI Enrichment (Snowflake Cortex)
- Sentiment analysis
- Summarization
- Semantic search

Using 2 AI SQL functions on the descriptive column in the
bronze layer table and store the output as additional outputs on bronze
layer table.

In [ ]:
USE SCHEMA BRONZE;

-- Drop columns if they exist (won't error if they don't exist)
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS DROP COLUMN IF EXISTS SENTIMENT_SCORE;
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS DROP COLUMN IF EXISTS SENTIMENT_LABEL;
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS DROP COLUMN IF EXISTS SUMMARY_TEXT;
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS DROP COLUMN IF EXISTS KEY_TOPICS;
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS DROP COLUMN IF EXISTS CLEANED_DESCRIPTION;

-- Add them fresh
-- Use maximum VARCHAR size in Snowflake for summary_text and key_topics
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS ADD COLUMN SENTIMENT_SCORE FLOAT;
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS ADD COLUMN SENTIMENT_LABEL VARCHAR(50);
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS ADD COLUMN SUMMARY_TEXT VARCHAR(16777216); 
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS ADD COLUMN KEY_TOPICS VARCHAR(16777216);
ALTER TABLE BRONZE.STEAM_DESCRIPTIONS ADD COLUMN CLEANED_DESCRIPTION VARCHAR(16777216);

In [ ]:
-- Remove HTML tags and clean up the text
UPDATE BRONZE.STEAM_DESCRIPTIONS
SET CLEANED_DESCRIPTION = TRIM(
    REGEXP_REPLACE(
        REGEXP_REPLACE(
            REGEXP_REPLACE(
                REGEXP_REPLACE(
                    REGEXP_REPLACE(
                        REGEXP_REPLACE("detailed_description", '<[^>]*>', ' '),  -- Remove HTML tags (pass 1)
                        '<[^>]*>', ' '                                           -- Remove HTML tags (pass 2 for nested)
                    ),
                    '&[a-zA-Z]+;|&#[0-9]+;', ' '                                -- Remove all HTML entities
                ),
                '\\"', ''                                                        -- Remove escaped quotes
            ),
            '[ \\t\\r\\n]+', ' '                                                 -- Replace multiple spaces with single space
        ),
        '(^[ ]+|[ ]+$)', ''                                                      -- Remove leading/trailing spaces
    )
)
WHERE "detailed_description" IS NOT NULL 
  AND TRIM("detailed_description") <> '';


SELECT CLEANED_DESCRIPTION
FROM BRONZE.STEAM_DESCRIPTIONS;

In [ ]:
-- AI Function 1: SENTIMENT Analysis
-- Analyze sentiment of detailed descriptions

UPDATE BRONZE.STEAM_DESCRIPTIONS
SET SENTIMENT_SCORE = SNOWFLAKE.CORTEX.SENTIMENT(CLEANED_DESCRIPTION)
WHERE CLEANED_DESCRIPTION IS NOT NULL 
  AND TRIM(CLEANED_DESCRIPTION) <> '';

-- Create sentiment labels based on score
UPDATE BRONZE.STEAM_DESCRIPTIONS
SET SENTIMENT_LABEL = CASE
    WHEN SENTIMENT_SCORE >= 0.5 THEN 'Very Positive'
    WHEN SENTIMENT_SCORE >= 0.1 THEN 'Positive'
    WHEN SENTIMENT_SCORE >= -0.1 THEN 'Neutral'
    WHEN SENTIMENT_SCORE >= -0.5 THEN 'Negative'
    ELSE 'Very Negative'
END
WHERE SENTIMENT_SCORE IS NOT NULL;


In [ ]:
-- AI Function 2: SUMMARIZE --
-- Generate concise summaries of detailed descriptions
-- Fixed short length (recommended for consistency)

UPDATE BRONZE.STEAM_DESCRIPTIONS
SET SUMMARY_TEXT = SNOWFLAKE.CORTEX.SUMMARIZE(CLEANED_DESCRIPTION)
WHERE CLEANED_DESCRIPTION IS NOT NULL
    AND TRIM(CLEANED_DESCRIPTION) <> ''
    AND LENGTH(CLEANED_DESCRIPTION) > 0;


In [ ]:
-- Verification Queries --

-- Check sentiment distribution
SELECT
    SENTIMENT_LABEL,
    COUNT(*) AS GAME_COUNT,
    ROUND(AVG(SENTIMENT_SCORE), 3) AS AVG_SCORE,
    ROUND(MIN(SENTIMENT_SCORE), 3) AS MIN_SCORE,
    ROUND(MAX(SENTIMENT_SCORE), 3) AS MAX_SCORE
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE SENTIMENT_LABEL IS NOT NULL
GROUP BY SENTIMENT_LABEL
ORDER BY AVG_SCORE DESC;

In [ ]:
-- Sample view of AI-enhanced descriptions
SELECT 
    "steam_appid",
    LEFT(CLEANED_DESCRIPTION, 200) AS DESCRIPTION_PREVIEW,
    SENTIMENT_SCORE,
    SENTIMENT_LABEL,
    LEFT(SUMMARY_TEXT, 500) AS SUMMARY,
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE SENTIMENT_SCORE IS NOT NULL
LIMIT 25;

In [ ]:
-- Compare original vs summarized lengths (length-wise)
SELECT 
    "steam_appid" AS APP_ID,
    LENGTH(CLEANED_DESCRIPTION) AS ORIGINAL_LENGTH,
    LENGTH("short_description") AS SHORT_DESC_LENGTH,
    LENGTH(SUMMARY_TEXT) AS SUMMARY_LENGTH,
    ROUND((LENGTH(SUMMARY_TEXT) / NULLIF(LENGTH(CLEANED_DESCRIPTION), 0)) * 100, 1) AS COMPRESSION_PCT
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE SUMMARY_TEXT IS NOT NULL
  AND CLEANED_DESCRIPTION IS NOT NULL
LIMIT 15;

In [ ]:
-- Compare original vs summarized lengths (word-wise)
SELECT 
    "steam_appid" AS APP_ID,
    CLEANED_DESCRIPTION AS ORIGINAL_LENGTH,
    "short_description" AS SHORT_DESC_LENGTH,
    SUMMARY_TEXT AS SUMMARY_LENGTH
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE SUMMARY_TEXT IS NOT NULL
  AND CLEANED_DESCRIPTION IS NOT NULL
LIMIT 15;

In [ ]:
-- Compare average lengths of the three description types --

-- Side-by-side comparison with min/max
SELECT 
    'CLEANED_DESCRIPTION' AS DESCRIPTION_TYPE,
    ROUND(AVG(LENGTH(CLEANED_DESCRIPTION)), 2) AS AVG_LENGTH,
    MIN(LENGTH(CLEANED_DESCRIPTION)) AS MIN_LENGTH,
    MAX(LENGTH(CLEANED_DESCRIPTION)) AS MAX_LENGTH,
    COUNT(*) AS ROW_COUNT
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE CLEANED_DESCRIPTION IS NOT NULL

UNION ALL

SELECT 
    'short_description' AS DESCRIPTION_TYPE,
    ROUND(AVG(LENGTH("short_description")), 2) AS AVG_LENGTH,
    MIN(LENGTH("short_description")) AS MIN_LENGTH,
    MAX(LENGTH("short_description")) AS MAX_LENGTH,
    COUNT(*) AS ROW_COUNT
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE "short_description" IS NOT NULL

UNION ALL

SELECT 
    'SUMMARY_TEXT' AS DESCRIPTION_TYPE,
    ROUND(AVG(LENGTH(SUMMARY_TEXT)), 2) AS AVG_LENGTH,
    MIN(LENGTH(SUMMARY_TEXT)) AS MIN_LENGTH,
    MAX(LENGTH(SUMMARY_TEXT)) AS MAX_LENGTH,
    COUNT(*) AS ROW_COUNT
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE SUMMARY_TEXT IS NOT NULL

ORDER BY AVG_LENGTH DESC;


There are certain rows with missing outputs in our new AI features. Let's analyze why that is.

In [ ]:
-- Count rows with missing AI outputs
SELECT 
    COUNT(*) AS TOTAL_ROWS,
    SUM(CASE WHEN CLEANED_DESCRIPTION IS NULL OR TRIM(CLEANED_DESCRIPTION) = '' THEN 1 ELSE 0 END) AS EMPTY_CLEANED,
    SUM(CASE WHEN SENTIMENT_SCORE IS NULL THEN 1 ELSE 0 END) AS MISSING_SENTIMENT,
    SUM(CASE WHEN SUMMARY_TEXT IS NULL THEN 1 ELSE 0 END) AS MISSING_SUMMARY
FROM BRONZE.STEAM_DESCRIPTIONS;

In [ ]:
-- Checks the remaining rows with null values

SELECT 
    "steam_appid",
    LENGTH(CLEANED_DESCRIPTION) AS CLEANED_LENGTH,
    SENTIMENT_SCORE,
    SUMMARY_TEXT,
    "detailed_description",
    "short_description"
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE SENTIMENT_SCORE IS NULL 
   OR SUMMARY_TEXT IS NULL
ORDER BY "steam_appid";

All the rows with missing features simply have images/html tags as their full detailed description, such that it got removed in the cleaning process and our AI function had nothing to summarize. Those games still have a short description that gives context, though summarizing an already short description would be redundant. 

In [ ]:
-- Find most positive games --
-- Descriptions with enthusiastic, exciting, upbeat language

SELECT 
    d."steam_appid" AS APP_ID,
    g."name" AS NAME,
    d.SENTIMENT_SCORE,
    d.SENTIMENT_LABEL,
    LEFT(d.SUMMARY_TEXT, 200) AS SUMMARY
FROM BRONZE.STEAM_DESCRIPTIONS d
JOIN BRONZE.STEAM_GAMES g ON d."steam_appid" = g."appid"
WHERE d.SENTIMENT_SCORE IS NOT NULL
ORDER BY d.SENTIMENT_SCORE DESC
LIMIT 10;

In [ ]:
-- Find most negative games --
-- Descriptions with dark, grim, or intense language

SELECT 
    d."steam_appid" AS APP_ID,
    g."name" AS NAME,
    d.SENTIMENT_SCORE,
    d.SENTIMENT_LABEL,
    LEFT(d.SUMMARY_TEXT, 200) AS SUMMARY
FROM BRONZE.STEAM_DESCRIPTIONS d
JOIN BRONZE.STEAM_GAMES g ON d."steam_appid" = g."appid"
WHERE d.SENTIMENT_SCORE IS NOT NULL
ORDER BY d.SENTIMENT_SCORE ASC
LIMIT 10;

#### Build a Cortex Search service to search on the descriptive column in the bronze layer and try few searches.

In [ ]:
-- Build Cortex Search Service --

USE SCHEMA BRONZE;

-- Create a view with uppercase column names for Cortex Search
-- Create the Cortex Search Service using the view
-- This indexes your CLEANED_DESCRIPTION column for fast semantic search
CREATE OR REPLACE VIEW BRONZE.VW_STEAM_DESCRIPTIONS_SEARCH AS
SELECT 
    "steam_appid" AS STEAM_APPID,
    CLEANED_DESCRIPTION
FROM BRONZE.STEAM_DESCRIPTIONS
WHERE CLEANED_DESCRIPTION IS NOT NULL
  AND TRIM(CLEANED_DESCRIPTION) <> '';

-- Create the Cortex Search Service using the view
CREATE OR REPLACE CORTEX SEARCH SERVICE STEAM_GAME_SEARCH
ON CLEANED_DESCRIPTION
ATTRIBUTES STEAM_APPID
WAREHOUSE = ANIMAL_TASK_WH
TARGET_LAG = 'DOWNSTREAM'
AS (
    SELECT 
        STEAM_APPID,
        CLEANED_DESCRIPTION
    FROM BRONZE.VW_STEAM_DESCRIPTIONS_SEARCH
);

-- Check status
SHOW CORTEX SEARCH SERVICES;

In [ ]:
-- Test Search: Horror games
SELECT 
    SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
        'BRONZE.STEAM_GAME_SEARCH',
        '{
            "query": "scary horror survival game with monsters",
            "columns": ["STEAM_APPID", "CLEANED_DESCRIPTION"],
            "limit": 10
        }'
    );

Results come back JSON so we need to parse them.

In [ ]:
-- Horror games --
-- Parse results and join with game info

WITH search_results AS (
    SELECT 
        SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
            'BRONZE.STEAM_GAME_SEARCH',
            '{
                "query": "scary horror survival game with monsters",
                "columns": ["STEAM_APPID"],
                "limit": 20
            }'
        ) AS results
)
SELECT 
    f.value:STEAM_APPID::NUMBER AS STEAM_APPID,
    ROUND(f.value:"@scores":cosine_similarity::FLOAT, 4) AS COSINE_SIMILARITY,
    ROUND(f.value:"@scores":text_match::FLOAT, 4) AS TEXT_MATCH_SCORE,
    g."name" AS GAME_NAME,
    LEFT(d.CLEANED_DESCRIPTION, 200) AS DESCRIPTION,
    d.SENTIMENT_LABEL,
    g."release_date" AS RELEASE_DATE
FROM search_results sr,
     LATERAL FLATTEN(input => PARSE_JSON(sr.results):results) f
JOIN BRONZE.STEAM_GAMES g ON f.value:STEAM_APPID::NUMBER = g."appid"
JOIN BRONZE.STEAM_DESCRIPTIONS d ON f.value:STEAM_APPID::NUMBER = d."steam_appid"
ORDER BY COSINE_SIMILARITY DESC
LIMIT 10;

In [ ]:
-- Adventure puzzle games --
-- Parse results and join with game info

WITH search_results AS (
    SELECT 
        SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
            'BRONZE.STEAM_GAME_SEARCH',
            '{
                "query": "adventure puzzle exploration",
                "columns": ["STEAM_APPID"],
                "limit": 20
            }'
        ) AS results
)
SELECT 
    results,
    -- Parse JSON structure
    g."name" AS GAME_NAME,
    LEFT(d.CLEANED_DESCRIPTION, 200) AS DESCRIPTION
FROM search_results sr,
     LATERAL FLATTEN(input => PARSE_JSON(sr.results):results) f
JOIN BRONZE.STEAM_GAMES g ON f.value:STEAM_APPID::NUMBER = g."appid"
JOIN BRONZE.STEAM_DESCRIPTIONS d ON f.value:STEAM_APPID::NUMBER = d."steam_appid"
LIMIT 10;

In [ ]:
-- Co-op shooter games --
-- Parse results and join with game info

WITH search_results AS (
    SELECT 
        SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
            'BRONZE.STEAM_GAME_SEARCH',
            '{
                "query": "cooperative shooting games",
                "columns": ["STEAM_APPID"],
                "limit": 20
            }'
        ) AS results
)
SELECT 
    results,
    -- Parse JSON structure
    g."name" AS GAME_NAME,
    LEFT(d.CLEANED_DESCRIPTION, 200) AS DESCRIPTION
FROM search_results sr,
     LATERAL FLATTEN(input => PARSE_JSON(sr.results):results) f
JOIN BRONZE.STEAM_GAMES g ON f.value:STEAM_APPID::NUMBER = g."appid"
JOIN BRONZE.STEAM_DESCRIPTIONS d ON f.value:STEAM_APPID::NUMBER = d."steam_appid"
LIMIT 10;

#### Natural Language Querying

Create denormalized view that joins all the relevant tables in the Silver layer. Cortex Analyst queries this view to get the data. Without it, there's no data to analyze.

In [ ]:
SELECT * FROM SILVER.FACT_GAME_METRICS;

In [ ]:
USE SCHEMA SILVER;

-- Create a denormalized view that joins all relevant tables
-- This makes it easier for Cortex Analyst to understand relationships
CREATE OR REPLACE VIEW SILVER.VW_GAME_ANALYTICS AS
SELECT 
    -- Game Information
    g.APP_ID,
    g.NAME AS GAME_NAME,
    g.RELEASE_DATE,
    g.REQUIRED_AGE,
    g.PLATFORMS,
    g.ENGLISH,
    
    -- Developer & Publisher
    dev.DEVELOPER AS DEVELOPER_NAME,
    pub.PUBLISHER AS PUBLISHER_NAME,
    
    -- Metrics
    f.PRICE,
    f.POSITIVE_RATINGS,
    f.NEGATIVE_RATINGS,
    f.POSITIVE_RATINGS + f.NEGATIVE_RATINGS AS TOTAL_RATINGS,
    CASE 
        WHEN f.NEGATIVE_RATINGS > 0 
        THEN ROUND(f.POSITIVE_RATINGS::FLOAT / f.NEGATIVE_RATINGS, 2)
        ELSE NULL 
    END AS RATING_RATIO,
    f.AVERAGE_PLAYTIME,
    f.MEDIAN_PLAYTIME,
    f.OWNERS,
    f.ACHIEVEMENTS,
    
    -- Categorization (comma-separated lists)
    LISTAGG(DISTINCT cat.CATEGORY, ', ') WITHIN GROUP (ORDER BY cat.CATEGORY) AS CATEGORIES,
    LISTAGG(DISTINCT gen.GENRE, ', ') WITHIN GROUP (ORDER BY gen.GENRE) AS GENRES

FROM SILVER.DIM_GAME g
LEFT JOIN SILVER.FACT_GAME_METRICS f ON g.APP_ID = f.APP_ID
LEFT JOIN SILVER.DIM_DEVELOPER dev ON f.DEVELOPER_KEY = dev.DEVELOPER_KEY
LEFT JOIN SILVER.DIM_PUBLISHER pub ON f.PUBLISHER_KEY = pub.PUBLISHER_KEY
LEFT JOIN SILVER.BRIDGE_GAME_CATEGORY bgc ON g.APP_ID = bgc.APP_ID
LEFT JOIN SILVER.DIM_CATEGORY cat ON bgc.CATEGORY_KEY = cat.CATEGORY_KEY
LEFT JOIN SILVER.BRIDGE_GAME_GENRE bgg ON g.APP_ID = bgg.APP_ID
LEFT JOIN SILVER.DIM_GENRE gen ON bgg.GENRE_KEY = gen.GENRE_KEY

GROUP BY 
    g.APP_ID, g.NAME, g.RELEASE_DATE, g.REQUIRED_AGE, g.PLATFORMS, g.ENGLISH,
    dev.DEVELOPER, pub.PUBLISHER,
    f.PRICE, f.POSITIVE_RATINGS, f.NEGATIVE_RATINGS, 
    f.AVERAGE_PLAYTIME, f.MEDIAN_PLAYTIME, f.OWNERS, f.ACHIEVEMENTS;

-- Test the view
SELECT * FROM SILVER.VW_GAME_ANALYTICS LIMIT 10;

Creating stage to upload YAML file

In [ ]:
-- Create a stage for the semantic model

USE DATABASE DB_TEAM_TRIPLEJ;
USE SCHEMA SILVER;

CREATE STAGE IF NOT EXISTS STEAM_SEMANTIC_STAGE;

-- Uploaded YAML file into stage
LIST @STEAM_SEMANTIC_STAGE;


### Uses the Snowflake UI Playground for Cortex Analyst.